# ME-IIS Colab

## Setup
- Clone repo if running in Colab
- Install requirements
- Set `PYTHONPATH`
- Print versions for reproducibility


In [ ]:

import os, sys, subprocess, json, time, datetime
import torch, sklearn, numpy as np
from importlib import import_module

REPO_ROOT = os.path.abspath('.')
if REPO_ROOT not in sys.path:
    sys.path.insert(0, REPO_ROOT)

print({'python': sys.version, 'torch': torch.__version__, 'sklearn': sklearn.__version__, 'numpy': np.__version__})


## Data configuration
- Expected dataset layout: Office-Home with domains under `Office-Home/{Art,Clipart,Product,Real_World}`.
- Provide `data_root` (e.g., from Google Drive) or use a small quick sanity subset if configured in the scripts via `--dry_run_*`.
- Notebook does **not** auto-download datasets; follow repository README for placement.


In [ ]:

# Global config for experiments (edit here)
config = {
    'dataset_name': 'office_home',
    'data_root': '/content/Office-Home',  # change to your path or mount
    'source_domain': 'Ar',
    'target_domain': 'Cl',
    'layers': 'layer3,layer4',
    'seed': 0,
    'quick_mode': True,  # set False for fuller runs
    'train_epochs': 1,
    'adapt_epochs': 1,
    'batch_size': 4,
    'gmm_selection_mode': 'fixed',
    'vmf_kappa': 20.0,
    'cluster_clean_ratio': 0.8,
}
print('Using config:', json.dumps(config, indent=2))


## Baseline experiments (existing)
Use existing cells below for source-only and ME-IIS with GMM. This notebook keeps all prior experiments intact; you can still run the old workflow.


## Experiments: vMF-softmax backend (spherical clustering) + entropy filtering
This section runs ME-IIS with the new `vmf_softmax` backend and optional entropy filtering, alongside source-only and GMM for comparison.


In [ ]:

from src.experiments.notebook_runner import (
    run_train,
    run_adapt,
    run_vmf_sweep,
    summarize_runs,
)

# Paths and common args
data_root = config['data_root']
common = {
    'dataset_name': config['dataset_name'],
    'data_root': data_root,
    'source_domain': config['source_domain'],
    'target_domain': config['target_domain'],
    'batch_size': config['batch_size'],
    'seed': config['seed'],
    'deterministic': True,
    'feature_layers': config['layers'],
}

# 1) Source-only
t_src = run_train(**common, num_epochs=config['train_epochs'], quick=config['quick_mode'])
print('Source-only checkpoint:', t_src['checkpoint'])

# 2) ME-IIS with GMM backend
gmm_run = run_adapt(
    checkpoint=t_src['checkpoint'],
    backend='gmm',
    gmm_selection_mode=config['gmm_selection_mode'],
    adapt_epochs=config['adapt_epochs'],
    quick=config['quick_mode'],
    **common,
)
print('GMM adapted checkpoint:', gmm_run['checkpoint'])

# 3) ME-IIS with vMF-softmax backend
vmf_run = run_adapt(
    checkpoint=t_src['checkpoint'],
    backend='vmf_softmax',
    vmf_kappa=config['vmf_kappa'],
    cluster_clean_ratio=config['cluster_clean_ratio'],
    adapt_epochs=config['adapt_epochs'],
    quick=config['quick_mode'],
    **common,
)
print('vMF adapted checkpoint:', vmf_run['checkpoint'])

summary = summarize_runs([gmm_run, vmf_run])
try:
    from IPython.display import display
    display(summary)
except Exception:
    print(summary)


### vMF hyperparameter sweep (grid)
Sweeps K, kappa, and cluster_clean_ratio for a fixed domain pair. Adjust `quick_mode` to keep runtime manageable.


In [ ]:

vmf_grid = {
    'K': [3, 6, 12],  # replace with [num_classes, 2*num_classes, 4*num_classes] if desired
    'kappa': [5, 10, 20, 40],
    'clean_ratio': [1.0, 0.8, 0.6],
}
vmf_results = run_vmf_sweep(
    base_checkpoint=t_src['checkpoint'],
    grid=vmf_grid,
    layers=config['layers'],
    adapt_epochs=config['adapt_epochs'],
    quick=config['quick_mode'],
    **common,
)
vmf_results_sorted = vmf_results.sort_values('adapted_target_acc', ascending=False).head(10)
try:
    from IPython.display import display
    display(vmf_results_sorted)
except Exception:
    print(vmf_results_sorted)


### Layer sensitivity (latent variable placement)
Compare different feature layer sets for both GMM and vMF backends.


In [ ]:

layer_sets = [
    'layer4',
    'layer3,layer4',
]
layer_rows = []
for layers in layer_sets:
    for backend in ['gmm', 'vmf_softmax']:
        run = run_adapt(
            checkpoint=t_src['checkpoint'],
            backend=backend,
            feature_layers=layers,
            vmf_kappa=config['vmf_kappa'],
            gmm_selection_mode=config['gmm_selection_mode'],
            adapt_epochs=config['adapt_epochs'],
            quick=config['quick_mode'],
            **{**common, 'feature_layers': layers},
        )
        row = {k: run.get(k) for k in ['backend', 'feature_layers', 'adapted_target_acc', 'delta_acc', 'iis_final_max_err', 'weight_entropy']}
        layer_rows.append(row)
import pandas as pd
layer_df = pd.DataFrame(layer_rows)
try:
    from IPython.display import display
    display(layer_df)
except Exception:
    print(layer_df)


## Results table + plots


In [ ]:

import pandas as pd
import matplotlib.pyplot as plt

if 'vmf_results' in globals() and not vmf_results.empty:
    plt.figure(figsize=(6,4))
    plt.scatter(vmf_results['kappa'], vmf_results['adapted_target_acc'], c=vmf_results['clean_ratio'], cmap='viridis')
    plt.xlabel('kappa'); plt.ylabel('adapted_target_acc'); plt.title('vMF sweep')
    plt.colorbar(label='clean_ratio')
    plt.show()


## Debugging / sanity checks
- Verify gamma responsibilities form valid PMFs.
- Verify joint feature mass per sample equals number of layers (N_c).
- Plot IIS convergence diagnostics if history is available.


In [ ]:

from clustering.factory import create_backend
from models.me_iis_adapter import MaxEntAdapter

backend = create_backend('vmf_softmax', n_components=3, seed=0, vmf_kappa=10.0)
X = np.array([[1.0, 0.0], [0.0, 1.0]], dtype=np.float64)
backend.fit(X)
gamma = backend.predict_proba(X)
print('gamma min', gamma.min(), 'row sums', gamma.sum(axis=1))

adapter = MaxEntAdapter(num_classes=2, layers=['l0','l1'], components_per_layer={'l0':3,'l1':3}, device=torch.device('cpu'))
class_probs = torch.full((2,2), 0.5, dtype=torch.float64)
resp = {'l0': torch.tensor(gamma, dtype=torch.float64), 'l1': torch.tensor(gamma, dtype=torch.float64)}
joint = adapter.joint_builder.build_joint_from_responsibilities(resp, class_probs)
flat, mass = adapter.joint_builder.validate_and_flatten(joint, rel_mass_tol=1e-6)
print('feature mass per sample:', mass)

if 'vmf_run' in globals() and vmf_run.get('history_npz'):
    import numpy as np
    data = np.load(vmf_run['history_npz'])
    if 'moment_max' in data:
        import matplotlib.pyplot as plt
        plt.figure(); plt.plot(data['moment_max']); plt.title('IIS max moment error'); plt.xlabel('iter'); plt.ylabel('error'); plt.show()


## Next steps / recommended sweeps
- Extend kappa/K sweep on full epochs when resources allow.
- Try entropy filtering ratios {1.0, 0.8, 0.6} for noisy targets.
- Compare layer sets including `layer2` if available.


<a href="https://colab.research.google.com/github/kaminglui/Domain-Adaptation-with-ME-IIS/blob/main/ME_IIS_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ME-IIS Domain Adaptation (Colab)
Single-pass pipeline to train a source ResNet-50 and adapt with ME-IIS. Edit the Section 3 config cell to switch datasets, domains, and options.

## 0. Setup & GPU check
- Mount Google Drive and choose a working directory.
- Clone/pull the repo into that folder.
- Confirm CUDA availability and show the GPU name.

In [9]:
# Section 0 - Setup & GPU check
import os
from pathlib import Path

USE_DRIVE = True  # Set False to keep everything in /content
DRIVE_ROOT = "/content/drive/MyDrive" if USE_DRIVE else "/content"
WORK_DIR = os.path.join(DRIVE_ROOT, "MEIIS-Colab")
REPO_URL = "https://github.com/kaminglui/Domain-Adaptation-with-ME-IIS.git"
REPO_NAME = "Domain-Adaptation-with-ME-IIS"
REPO_DIR = os.path.join(WORK_DIR, REPO_NAME)

if USE_DRIVE:
    from google.colab import drive  # type: ignore
    drive.mount("/content/drive")

os.makedirs(WORK_DIR, exist_ok=True)
os.chdir(WORK_DIR)
print("WORK_DIR:", WORK_DIR)

if not os.path.isdir(REPO_DIR):
    print("[Repo] Cloning repository...")
    !git clone {REPO_URL}
else:
    print("[Repo] Repository exists; pulling latest changes...")
    os.chdir(REPO_DIR)
    !git pull
    os.chdir(WORK_DIR)

os.chdir(REPO_DIR)
print("[Repo] Current repo dir:", os.getcwd())

try:
    import torch
    print("torch.cuda.is_available():", torch.cuda.is_available())
    if torch.cuda.is_available():
        print("GPU:", torch.cuda.get_device_name(0))
    else:
        print("GPU not detected - switch runtime to GPU for full runs.")
except ImportError as exc:
    print("PyTorch not installed yet; run Section 1 to install dependencies. GPU check skipped:", exc)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
WORK_DIR: /content/drive/MyDrive/MEIIS-Colab
[Repo] Repository exists; pulling latest changes...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0 (from 0)
Unpacking objects: 100% (3/3), 553 bytes | 21.00 KiB/s, done.
From https://github.com/kaminglui/Domain-Adaptation-with-ME-IIS
   69fcd02..375cdfd  main       -> origin/main
Updating 69fcd02..375cdfd
Fast-forward
 ME_IIS_Colab.ipynb | 30 ++++--------------------------
 1 file changed, 4 insertions(+), 26 deletions(-)
[Repo] Current repo dir: /content/drive/MyDrive/MEIIS-Colab/Domain-Adaptation-with-ME-IIS
torch.cuda.is_available(): True
GPU: NVIDIA L4


## 1. Install dependencies
Re-run this cell if the Colab runtime restarts.

In [10]:
# Section 1 - Install dependencies (re-run if the runtime restarts)
import os

os.chdir(REPO_DIR)
req_file = "env/requirements_colab.txt" if os.path.exists("env/requirements_colab.txt") else "env/requirements.txt"
print("Installing dependencies from:", req_file)
!pip install -r {req_file}


Installing dependencies from: env/requirements_colab.txt


## 2. Download datasets via KaggleHub
Downloads Office-Home and Office-31 with KaggleHub, locates the canonical roots, and links them under `datasets/` for the scripts.

In [11]:
# Section 2 - Download datasets via KaggleHub
import os
import pathlib

os.chdir(REPO_DIR)
print("Repo dir:", os.getcwd())

try:
    import kagglehub  # type: ignore
except ImportError:
    print("[Data] Installing kagglehub...")
    !pip install kagglehub
    import kagglehub  # type: ignore

def _find_office_home_root(base_dir: pathlib.Path) -> pathlib.Path:
    candidates = [base_dir] + [p for p in base_dir.rglob("*") if p.is_dir()]
    for cand in candidates:
        names = {p.name for p in cand.iterdir() if p.is_dir()}
        if {"Art", "Clipart", "Product"} <= names and any(n.lower().startswith("real") for n in names):
            return cand
    return base_dir

def _find_office31_root(base_dir: pathlib.Path) -> pathlib.Path:
    candidates = [base_dir] + [p for p in base_dir.rglob("*") if p.is_dir()]
    for cand in candidates:
        names = {p.name.lower() for p in cand.iterdir() if p.is_dir()}
        if {"amazon", "dslr", "webcam"} <= names:
            return cand
    return base_dir

print("[Data] Downloading Office-Home (lhrrraname/officehome)...")
office_home_raw = pathlib.Path(kagglehub.dataset_download("lhrrraname/officehome"))
office_home_root = _find_office_home_root(office_home_raw)
print("  Office-Home root:", office_home_root)

print("[Data] Downloading Office-31 (xixuhu/office31)...")
office31_raw = pathlib.Path(kagglehub.dataset_download("xixuhu/office31"))
office31_root = _find_office31_root(office31_raw)
print("  Office-31 root:", office31_root)

datasets_dir = pathlib.Path("datasets")
datasets_dir.mkdir(exist_ok=True)

def _ensure_link(link_path: pathlib.Path, target: pathlib.Path) -> None:
    target = target.resolve()
    if link_path.exists() and not link_path.is_symlink():
        print(f"[Data] {link_path} exists and is not a symlink; leaving as-is.")
        return
    if link_path.is_symlink():
        current = link_path.resolve()
        if current == target:
            print(f"[Data] {link_path} already points to {target}")
            return
        link_path.unlink()
    try:
        link_path.symlink_to(target, target_is_directory=True)
        print(f"[Data] Linked {link_path} -> {target}")
    except OSError as exc:
        print(f"[Data] Could not create symlink {link_path} -> {target}: {exc}")

_ensure_link(datasets_dir / "Office-Home", office_home_root)
_ensure_link(datasets_dir / "Office-31", office31_root)

OFFICE_HOME_ROOT = datasets_dir / "Office-Home"
OFFICE31_ROOT = datasets_dir / "Office-31"
print("[Data] Office-Home DATA_ROOT:", OFFICE_HOME_ROOT)
print("[Data] Office-31 DATA_ROOT:", OFFICE31_ROOT)


Repo dir: /content/drive/MyDrive/MEIIS-Colab/Domain-Adaptation-with-ME-IIS
[Data] Downloading Office-Home (lhrrraname/officehome)...
Using Colab cache for faster access to the 'officehome' dataset.
  Office-Home root: /kaggle/input/officehome/datasets/OfficeHomeDataset_10072016
[Data] Downloading Office-31 (xixuhu/office31)...
Using Colab cache for faster access to the 'office31' dataset.
  Office-31 root: /kaggle/input/office31/Office-31
[Data] Linked datasets/Office-Home -> /kaggle/input/officehome/datasets/OfficeHomeDataset_10072016
[Data] Linked datasets/Office-31 -> /kaggle/input/office31/Office-31
[Data] Office-Home DATA_ROOT: datasets/Office-Home
[Data] Office-31 DATA_ROOT: datasets/Office-31


## 3. Configuration (single cell with all knobs)
Edit only in this cell to change experiment settings.

In [12]:

# Edit only in this cell to change experiment settings.

# Dataset & domains
DATASET_NAME = "office_home"  # or "office31"
SOURCE_DOMAIN = "Ar"          # e.g. Ar/Cl/Pr/Rw for Office-Home, A/D/W for Office-31
TARGET_DOMAIN = "Cl"
SEED = 0

# Paths (assume we are in the repo root)
DATA_ROOT = "datasets/Office-Home"  # or "datasets/Office-31"

# Source training hyperparameters
NUM_EPOCHS_SRC = 100
BATCH_SIZE_SRC = 32
LR_BACKBONE = 1e-3
LR_CLASSIFIER = 1e-2
WEIGHT_DECAY = 1e-3
NUM_WORKERS = 4
DETERMINISTIC = True  # set True to minimize randomness

# ME-IIS / adaptation hyperparameters
# We found that using layer3+layer4 preserves spatial/style information.
# Including avgpool (global pooled features) sharply reduced performance,
# so we disable it by default. You can add it back by changing this string.
FEATURE_LAYERS = "layer3,layer4"
GMM_SELECTION_MODE = "bic"        # "fixed" or "bic"
NUM_LATENT_STYLES = 5             # default components per layer (fixed mode or BIC init)
GMM_BIC_MIN_COMPONENTS = 2        # BIC lower bound
GMM_BIC_MAX_COMPONENTS = 8        # BIC upper bound

# NEW: source_prob_mode
SOURCE_PROB_MODE = "softmax"      # or "onehot" (use GT labels for source constraints)

# NEW: optional per-layer GMM override
# e.g. "layer3:10,layer4:5" or "7,8,9" for 3 layers; leave "" to disable
COMPONENTS_OVERRIDE = ""

# Pseudo-label adaptation (ME-IIS+PL)
USE_PSEUDO_LABELS = False         # base flag for direct runs
PSEUDO_CONF_THRESH = 0.9
PSEUDO_MAX_RATIO = 1.0            # 1.0 = no cap; or e.g. 0.3 to limit pseudo-target count
PSEUDO_LOSS_WEIGHT = 1.0

# IIS/Adapt config
IIS_ITERS = 15
IIS_TOL = 1e-3
ADAPT_EPOCHS = 10
FINETUNE_BACKBONE = False
BACKBONE_LR_SCALE = 0.1
CLASSIFIER_LR = 1e-2
ADAPT_WEIGHT_DECAY = 1e-3

# NEW: warm start for pseudo-labels
WARMUP_EPOCHS_NO_PL = 5   # Stage-1 epochs with no pseudo-labels
STAGE2_EPOCHS_PL = 10     # Stage-2 epochs with pseudo-labels

# Optional: force fresh runs instead of auto-resume
FORCE_FRESH_SOURCE_TRAIN = False
FORCE_FRESH_ADAPT = False


## 4. Helper: construct checkpoint paths and cleanup
Lets you intentionally bypass auto-resume by deleting existing checkpoints.

In [13]:
import os

os.chdir(REPO_DIR)

def get_source_ckpt_path():
    return f"checkpoints/source_only_{SOURCE_DOMAIN}_to_{TARGET_DOMAIN}_seed{SEED}.pth"

def get_me_iis_ckpt_path():
    layers_str = FEATURE_LAYERS.replace(",", "-").replace(" ", "")
    return f"checkpoints/me_iis_{SOURCE_DOMAIN}_to_{TARGET_DOMAIN}_{layers_str}_seed{SEED}.pth"

if FORCE_FRESH_SOURCE_TRAIN:
    src_ckpt = get_source_ckpt_path()
    if os.path.exists(src_ckpt):
        os.remove(src_ckpt)
        print("Deleted existing source checkpoint:", src_ckpt)

if FORCE_FRESH_ADAPT:
    adapt_ckpt = get_me_iis_ckpt_path()
    if os.path.exists(adapt_ckpt):
        os.remove(adapt_ckpt)
        print("Deleted existing ME-IIS checkpoint:", adapt_ckpt)


## 5. Train source-only model (full training)
Build the command from the config above and stream output directly in the cell.

In [14]:
import os

det_flag = "--deterministic" if DETERMINISTIC else ""
os.chdir(REPO_DIR)

SRC_CMD = (
    'python scripts/train_source.py \
'
    f'  --dataset_name {DATASET_NAME} \
'
    f'  --data_root "{DATA_ROOT}" \
'
    f'  --source_domain {SOURCE_DOMAIN} \
'
    f'  --target_domain {TARGET_DOMAIN} \
'
    f'  --num_epochs {NUM_EPOCHS_SRC} \
'
    f'  --batch_size {BATCH_SIZE_SRC} \
'
    f'  --lr_backbone {LR_BACKBONE} \
'
    f'  --lr_classifier {LR_CLASSIFIER} \
'
    f'  --weight_decay {WEIGHT_DECAY} \
'
    f'  --num_workers {NUM_WORKERS} \
'
    f'  {det_flag} \
'
    f'  --seed {SEED}'
).strip()

print("Training command:\n", SRC_CMD)
!{SRC_CMD}


Training command:
 python scripts/train_source.py   --dataset_name office_home   --data_root "datasets/Office-Home"   --source_domain Ar   --target_domain Cl   --num_epochs 100   --batch_size 32   --lr_backbone 0.001   --lr_classifier 0.01   --weight_decay 0.001   --num_workers 4   --deterministic   --seed 0
2025-12-10 06:33:50.236055: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-10 06:33:50.254306: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765348430.276370    4114 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00

## 6. Run ME-IIS adaptation
Uses the source checkpoint above plus the ME-IIS settings in the config cell.

In [16]:

import os
from pathlib import Path

os.chdir(REPO_DIR)

def build_adapt_cmd(use_pseudo: bool, adapt_epochs: int, extra_tag: str = ""):
    src_ckpt = get_source_ckpt_path()
    det_flag = "--deterministic" if DETERMINISTIC else ""
    cmd = f"""
    python scripts/adapt_me_iis.py       --dataset_name {DATASET_NAME}       --data_root "{DATA_ROOT}"       --source_domain {SOURCE_DOMAIN}       --target_domain {TARGET_DOMAIN}       --checkpoint {src_ckpt}       --batch_size {BATCH_SIZE_SRC}       --num_workers {NUM_WORKERS}       --feature_layers "{FEATURE_LAYERS}"       --num_latent_styles {NUM_LATENT_STYLES}       --gmm_selection_mode {GMM_SELECTION_MODE}       --gmm_bic_min_components {GMM_BIC_MIN_COMPONENTS}       --gmm_bic_max_components {GMM_BIC_MAX_COMPONENTS}       --iis_iters {IIS_ITERS}       --iis_tol {IIS_TOL}       --adapt_epochs {adapt_epochs}       {'--finetune_backbone' if FINETUNE_BACKBONE else ''}       --backbone_lr_scale {BACKBONE_LR_SCALE}       --classifier_lr {CLASSIFIER_LR}       --weight_decay {ADAPT_WEIGHT_DECAY}       --source_prob_mode {SOURCE_PROB_MODE}       --seed {SEED}       {det_flag}
    """
    if COMPONENTS_OVERRIDE.strip():
        cmd += f' --components_per_layer "{COMPONENTS_OVERRIDE.strip()}" '

    if use_pseudo:
        cmd += (
            f" --use_pseudo_labels --pseudo_conf_thresh {PSEUDO_CONF_THRESH} "
            f"--pseudo_max_ratio {PSEUDO_MAX_RATIO} --pseudo_loss_weight {PSEUDO_LOSS_WEIGHT}"
        )

    return cmd.strip()

ran_stage = False

# Stage 1: warm-up ME-IIS without pseudo-labels
if WARMUP_EPOCHS_NO_PL > 0:
    ran_stage = True
    print(f"Running ME-IIS warm-up without pseudo-labels for {WARMUP_EPOCHS_NO_PL} epochs...")
    ADAPT_CMD_STAGE1 = build_adapt_cmd(use_pseudo=False, adapt_epochs=WARMUP_EPOCHS_NO_PL, extra_tag="_stage1")
    print("Stage-1 command:" + ADAPT_CMD_STAGE1)
    print("Stage-1 starts from the source-only checkpoint and writes the standard ME-IIS checkpoint.")
    !{ADAPT_CMD_STAGE1}

# Stage 2: ME-IIS with pseudo-labels
if STAGE2_EPOCHS_PL > 0:
    ran_stage = True
    print(f"Running ME-IIS with pseudo-labels for {STAGE2_EPOCHS_PL} epochs...")
    ADAPT_CMD_STAGE2 = build_adapt_cmd(use_pseudo=True, adapt_epochs=STAGE2_EPOCHS_PL, extra_tag="_stage2")
    print("Stage-2 command:" + ADAPT_CMD_STAGE2)
    print(
        "Stage-2 also points to the source-only checkpoint; adapt_me_iis.py will auto-resume from "
        "any existing ME-IIS checkpoint (e.g., the Stage-1 weights) and overwrite it with the pseudo-label run."
    )
    !{ADAPT_CMD_STAGE2}

# Fallback: single-stage adaptation (original behavior)
if not ran_stage:
    print("Running single-stage adaptation (no warm start configured)...")
    ADAPT_CMD = build_adapt_cmd(use_pseudo=USE_PSEUDO_LABELS, adapt_epochs=ADAPT_EPOCHS)
    print("Adaptation command:" + ADAPT_CMD)
    !{ADAPT_CMD}


Running ME-IIS warm-up without pseudo-labels for 5 epochs...
Stage-1 command:python scripts/adapt_me_iis.py       --dataset_name office_home       --data_root "datasets/Office-Home"       --source_domain Ar       --target_domain Cl       --checkpoint checkpoints/source_only_Ar_to_Cl_seed0.pth       --batch_size 32       --num_workers 4       --feature_layers "layer3,layer4"       --num_latent_styles 5       --gmm_selection_mode bic       --gmm_bic_min_components 2       --gmm_bic_max_components 8       --iis_iters 15       --iis_tol 0.001       --adapt_epochs 5              --backbone_lr_scale 0.1       --classifier_lr 0.01       --weight_decay 0.001       --source_prob_mode softmax       --seed 0       --deterministic
Stage-1 starts from the source-only checkpoint and writes the standard ME-IIS checkpoint.
2025-12-10 06:35:09.730024: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off err

## 7. Optional: experiment driver examples
Leave `RUN_EXPERIMENT_DRIVER = False` for the standard train + adapt path.

In [17]:
RUN_EXPERIMENT_DRIVER = True

if RUN_EXPERIMENT_DRIVER:
    import os
    os.chdir(REPO_DIR)
    det_flag = "--deterministic" if DETERMINISTIC else ""
    EXP_CMD = (
        'python scripts/run_me_iis_experiments.py \
'
        f'  --dataset_name {DATASET_NAME} \
'
        f'  --source_domain {SOURCE_DOMAIN} \
'
        f'  --target_domain {TARGET_DOMAIN} \
'
        '  --experiment_family layers \
'
        f'  --seeds {SEED} \
'
        f'  --base_data_root "{DATA_ROOT}" \
'
        f'  --feature_layers "{FEATURE_LAYERS}" \
'
        f'  --num_latent_styles {NUM_LATENT_STYLES} \
'
        f'  --gmm_selection_mode {GMM_SELECTION_MODE} \
'
        f'  --gmm_bic_min_components {GMM_BIC_MIN_COMPONENTS} \
'
        f'  --gmm_bic_max_components {GMM_BIC_MAX_COMPONENTS} \
'
        f'  --source_prob_mode {SOURCE_PROB_MODE} \
'
        f'  --num_epochs {NUM_EPOCHS_SRC} \
'
        f'  --batch_size {BATCH_SIZE_SRC} \
'
        f'  --num_workers {NUM_WORKERS} \
'
        f'  --iis_iters {IIS_ITERS} \
'
        f'  --iis_tol {IIS_TOL} \
'
        f'  --adapt_epochs {ADAPT_EPOCHS} \
'
        f'  --backbone_lr_scale {BACKBONE_LR_SCALE} \
'
        f'  --classifier_lr {CLASSIFIER_LR} \
'
        f'  --weight_decay {ADAPT_WEIGHT_DECAY} \
'
        f'  {'--finetune_backbone' if FINETUNE_BACKBONE else ''} \
'
        f'  {det_flag} \
'
        f'  --output_csv results/me_iis_layers_{SOURCE_DOMAIN}_to_{TARGET_DOMAIN}_seed{SEED}.csv'
    ).strip()

    if COMPONENTS_OVERRIDE.strip():
        EXP_CMD += f' --components_per_layer "{COMPONENTS_OVERRIDE}"'

    if USE_PSEUDO_LABELS:
        EXP_CMD += (
            f" --pseudo_conf_thresh {PSEUDO_CONF_THRESH} "
            f"--pseudo_max_ratio {PSEUDO_MAX_RATIO} --pseudo_loss_weight {PSEUDO_LOSS_WEIGHT}"
        )

    if det_flag:
        EXP_CMD += f" {det_flag}"

    print("Experiment driver command:\n", EXP_CMD)
    !{EXP_CMD}
else:
    print("Experiment driver is disabled (set RUN_EXPERIMENT_DRIVER = True to run).")


Experiment driver command:
 python scripts/run_me_iis_experiments.py   --dataset_name office_home   --source_domain Ar   --target_domain Cl   --experiment_family layers   --seeds 0   --base_data_root "datasets/Office-Home"   --feature_layers "layer3,layer4"   --num_latent_styles 5   --gmm_selection_mode bic   --gmm_bic_min_components 2   --gmm_bic_max_components 8   --source_prob_mode softmax   --num_epochs 100   --batch_size 32   --num_workers 4   --iis_iters 15   --iis_tol 0.001   --adapt_epochs 10   --backbone_lr_scale 0.1   --classifier_lr 0.01   --weight_decay 0.001      --deterministic   --output_csv results/me_iis_layers_Ar_to_Cl_seed0.csv --deterministic
2025-12-10 06:40:10.161393: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-10 06:40:10.179025: E exter

## 8. Notes on outputs
- Source checkpoints: `checkpoints/source_only_*` (auto-resume uses the matching file name).
- ME-IIS checkpoints and IIS weights: `checkpoints/me_iis_*` and `results/me_iis_weights_*.npz`.
- CSV log of source/adapt runs: `results/office_home_me_iis.csv` (dataset column distinguishes Office-Home vs Office-31).
- Experiment driver summaries (optional): `results/me_iis_experiments_summary.csv` or the path you pass via `--output_csv`.
- TensorBoard logs live under `runs/` (`runs/source_only` and `runs/adapt_me_iis`).